# FOOD AND BEVERAGE BUSINESS OPPORTUNITY SEEKER

This is an independent project centered around data science application in food and beverage industry. 

## 1. Introduction

There is a businessman lives in a capital city of Indonesia, Jakarta. He wants to start a food and beverage business in the city where he is living in, but he doesn't know how to start. Without any experience in this kind of business, he comes to us and tells us about his plan. He hopes that you can help him to make a right decision. As a Data Scientist, you are challenged to support him by giving him best recommendation in order to build a sucessful business.

This new businessman has two critical questions here, they are:
1. What kind of food and beverage business that he should start?
2. Where in the city should he start this business?

## 2. Data

In order to answer those two questions, as a data scientist, we need to have data that can be used as  